In [17]:
%%javascript
require.config({
   paths: {cytoscape: 'http://localhost:8099/js/cytoscape-2.7.10'}
   })

<IPython.core.display.Javascript object>

In [18]:
# ensure the library is available
import requests
r = requests.get('http://localhost:8099/js/cytoscape-2.7.10.js')
assert(r.status_code == 200)

In [19]:
import re

In [20]:
import igraph as ig
DEF_SCALING = 100.0
def from_igraph(igraph_network, layout=None, scale=DEF_SCALING):
    new_graph = {}
    network_data = {}
    elements = {}
    nodes = []
    edges = []
    network_attr = igraph_network.attributes() # Convert network attributes
    for key in network_attr:
        network_data[key] = igraph_network[key]
    edges_original = igraph_network.es # get network as a list of edges
    nodes_original = igraph_network.vs
    node_attr = igraph_network.vs.attributes()
    for idx, node in enumerate(nodes_original):
        new_node = {}
        data = {}
        data['id'] = str(node.index)
        data['name'] = str(node.index)
        for key in node_attr:
            data[key] = node[key]
        new_node['data'] = data
        if layout is not None:
            position = {}
            position['x'] = layout[idx][0] * scale
            position['y'] = layout[idx][1] * scale
            new_node['position'] = position
        nodes.append(new_node)
    edge_attr = igraph_network.es.attributes()  # Add edges to the elements
    for edge in edges_original:
        new_edge = {}
        data = {}
        data['source'] = str(edge.source)
        data['target'] = str(edge.target)
        for key in edge_attr:
            data[key] = edge[key]
        new_edge['data'] = data
        edges.append(new_edge)
    elements['nodes'] = nodes
    elements['edges'] = edges
    new_graph['elements'] = elements
    new_graph['data'] = network_data
    return new_graph


def to_igraph(network):

    nodes = network['elements']['nodes']
    edges = network['elements']['edges']
    network_attr = network['data']

    node_count = len(nodes)
    edge_count = len(edges)

    g = ig.Graph()

    # Graph attributes
    for key in network_attr.keys():
        g[key] = network_attr[key]

    g.add_vertices(nodes)

    # Add node attributes
    node_attributes = {}
    node_id_dict = {}
    for i, node in enumerate(nodes):
        data = node['data']
        for key in data.keys():
            if key not in node_attributes:
                node_attributes[key] = [None] * node_count

            # Save index to map
            if key == 'id':
                node_id_dict[data[key]] = i

            node_attributes[key][i] = data[key]

    for key in node_attributes.keys():
        g.vs[key] = node_attributes[key]

    # Create edges
    edge_tuples = []
    edge_attributes = {}
    for i, edge in enumerate(edges):
        data = edge['data']
        source = data['source']
        target = data['target']
        edge_tuple = (node_id_dict[source], node_id_dict[target])
        edge_tuples.append(edge_tuple)
        for key in data.keys():
            if key not in edge_attributes:
                edge_attributes[key] = [None] * edge_count

            # Save index to map
            edge_attributes[key][i] = data[key]

    g.add_edges(edge_tuples)

    # Assign edge attributes
    for key in edge_attributes.keys():
        if key == 'source' or key == 'target':
            continue
        else:
            g.es[key] = edge_attributes[key]

    return g


In [21]:
import ipywidgets as widgets
import json
import time
import os.path
from IPython.display import display, HTML
from traitlets import Int, Unicode, Tuple, CInt, Dict, validate, observe

class cyjs_sa(widgets.DOMWidget):
    
    _view_name = Unicode('CyjsView').tag(sync=True)
    _view_module = Unicode('cyjs_sa').tag(sync=True)
    frameHeight = Int(300).tag(sync=True)
    msgFromKernel = Unicode("{}").tag(sync=True)
    msgFromBrowser = Unicode("{}").tag(sync=True)
    _incomingMessage = {};
    status = "initial status message\n"
    _selectedNodes = [];
    
        
    def addGraph(self, g):
      self._resetMessage();
      gjson = from_igraph(g)
      self.msgFromKernel = json.dumps({"cmd": "addGraph", "status": "request",
                                       "callback": "",    "payload": gjson});
     
    def addGraphWithLayout(self, g, layout):
      self._resetMessage();
      gjson = from_igraph(g, layout)
      self.msgFromKernel = json.dumps({"cmd": "addGraph", "status": "request",
                                       "callback": "",    "payload": gjson});
     
    def deleteGraph(self):
      self._resetMessage();
      self.msgFromKernel = json.dumps({"cmd": "deleteGraph", "status": "request", "callback": "", "payload": ""});
     
    
    def setHeight(self, newHeight):
      self.frameHeight = newHeight
        
    def fitSelected(self, margin=50):
      self.status += "entering fitSelected (%d)\n" % margin
      self.msgFromKernel = json.dumps({"cmd": "fitSelected", "status": "request", "callback": "", "payload": margin});
        
    def fit(self, margin=50):
      self.status += "entering fit (%d)\n" % margin
      self.msgFromKernel = json.dumps({"cmd": "fit", "status": "request", "callback": "", "payload": margin});
        
    def getSelectedNodes(self):
       return(self._selectedNodes);

    def selectNodes(self, nodes):
      self.msgFromKernel = json.dumps({"cmd": "selectNodes", "status": "request",
                                       "callback": "", "payload": nodes});

    def _resetMessage(self):   # ensures that any ensuing message is seen as novel in the browser
       self.msgFromKernel = json.dumps({"cmd": "cleanSlate", "status": "nop", "callback": "", "payload": ""});


    def availableLayouts(self):
        return(["grid", "null", "random", "cose", "circle", "concentric", "breadthfirst"]);
    
    def setPosition(self, igraph_layout):   # the somewhat cryptic object created by igraph layouts
      tblPos = []
      for i in range(0, len(igraph_layout)):
         x = igraph_layout[i][0] * 100
         y = igraph_layout[i][1] * 100
         tblPos.append({"id": i, "x": x, "y": y})
      self._resetMessage();
      self.msgFromKernel = json.dumps({"cmd": "setPosition", "status": "request", "callback": "", 
                                       "payload": tblPos});
       
    def loadStyleFile(self, filename):
      if(not os.path.isfile(filename)):
        print("file '%s' not found" % filename)
        return;
      self._resetMessage();
      self.msgFromKernel = json.dumps({"cmd": "loadStyleFile", "status": "request", "callback": "", 
                                       "payload": filename});
        
    def layout(self, name):
      self._resetMessage();
      self.msgFromKernel = json.dumps({"cmd": "layout", "status": "request", "callback": "", "payload": name});
        
        
    def clearSelection(self):
      self._resetMessage();
      self. msgFromKernel = json.dumps({"cmd": "clearSelection", "status": "request",
                                        "callback": "", "payload": ""});
        
    def setNodeAttributes(self, attributeName, nodeNames, values):
      self._resetMessage();
      self. msgFromKernel = json.dumps({"cmd": "setNodeAttributes", "status": "request",
                                        "callback": "", 
                                        "payload": {"attributeName": attributeName,
                                                    "nodeNames": nodeNames,
                                                    "values": values}})

    def setEdgeAttributes(self, attributeName, sourceNames, targetNames, edgeTypes, values):
        # g to be a class member variable?
        # nodeMap also, with vigilance for modification of underlying graph?
        # id lookup a member function?
      names = g.vs['name']
      ids = [g.vs.find(name).index for name in g.vs['name']]
      nodeMap = dict(zip(names, ids))
      sourceIDs = [nodeMap[name] for name in sourceNames]
      targetIDs = [nodeMap[name] for name in targetNames]
      self._resetMessage(); 
      self. msgFromKernel = json.dumps({"cmd": "setEdgeAttributes", "status": "request",
                                        "callback": "", 
                                        "payload": {"attributeName": attributeName,
                                                    "sourceNames": sourceIDs,
                                                    "targetNames": targetIDs,
                                                    "edgeTypes": edgeTypes,
                                                    "values": values}})

    @observe('msgFromBrowser')
    def msg_arrived(self, change):
       self.status += "msgFromBrowser has arrived: %f\n" % time.time()
       rawMessage = change['new']
       self._incomingMessage = json.loads(rawMessage)
       cmd = self._incomingMessage["cmd"]
       self.status += "msg: %s\n"  % cmd
       if(cmd == "updateSelectedNodes"):
          self._selectedNodes = self._incomingMessage["payload"]
         
    def getResponse(self):
       return(self._incomingMessage["payload"])

    def getFullResponse(self):
       return(self._incomingMessage)
        

In [22]:
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [37]:
%%javascript
"use strict";

require.undef('cyjs_sa');

define('cyjs_sa', ["jupyter-js-widgets", "cytoscape"], function(widgets, cytoscape) {
    
    var CyjsView = widgets.DOMWidgetView.extend({

        initialize: function() {
           console.log("constructing CyjsView");
           this.frameHeight = "300px";
           this.options = {}; 
           this.msg = "empty in javascript";
           this.msgFromKernel = "";
           },

        resizeHandler: function(){
           console.log("cyjs resizeHandler")            
           },

        createDiv: function(){
            var outerDiv = $("<div id='cyOuterDiv' style='border:1px solid gray; height: 400px; width: 97%'></div>");
            var toolbarDiv = $("<div id='cyToolbarDiv' style='height: 30px; width: 97%'></div>");
            var cyDiv = $("<div id='cyDiv' style='height: 100%; width: 97%'></div>");
            outerDiv.append(toolbarDiv);
            outerDiv.append(cyDiv);
            var cyWidget = this;
            var fitButton = $("<button>Fit</button>").click(function(){
                console.log("Fit!");
                console.log("fitButton's notion of this:")
                console.log(cyWidget.cy);
                cyWidget.cy.fit(50);
               });
            toolbarDiv.append(fitButton);
            var fitSelectedButton = $("<button>Fit Selected</button>").click(function(){
                var selectedNodes = cyWidget.cy.filter('node:selected');
                if(selectedNodes.length > 0){
                   cyWidget.cy.fit(selectedNodes, 50);
                   }
               });
            toolbarDiv.append(fitSelectedButton);
            var sfnButton = $("<button>SFN</button>").click(function(){
               cyWidget.cy.nodes(':selected').neighborhood().nodes().select()
               });
            toolbarDiv.append(sfnButton);
            var clearButton = $("<button>Clear</button>").click(function(){
               cyWidget.cy.nodes().unselect();
               cyWidget.cy.edges().unselect();
               });
            toolbarDiv.append(clearButton);
            $(window).resize(this.resizeHandler);
            return(outerDiv);
           },
 
        
        createCanvas: function(){
            var cyjsWidget = this;
            console.log("createCanvas notion of this:")
            console.log(cyjsWidget);
            this.cy = cytoscape({
               container: document.getElementById('cyDiv'),
               layout: {name: 'preset'},
          ready: function(){
            console.log("small cyjs network ready");
            console.log("ready's notion of this:")
            console.log(this);
            cyjsWidget.cy = this;
            window.cy = this;  // for easy debugging
            console.log("ready's notion of cyjsWidget:")
            console.log(cyjsWidget);
            console.log("calling this.fit")
            //cyWidget.cy.fit(100);
            console.log("--- about to call loadGraph")
            //cyjsWidget.loadGraph("priyanka.json");
            cyjsWidget.loadStyle("style.js");
            //cyjsWidget.cy.layout({"name": "grid"});
            console.log("    back from loadGraph")
            cyjsWidget.cy.on("select", function(){
               console.log("calling updateSelectionToKernel, on select");
               cyjsWidget.updateSelectionToKernel(cyjsWidget);
               });
            cyjsWidget.cy.on("unselect", function(){
               console.log("calling updateSelectionToKernel, on unselect");
               cyjsWidget.updateSelectionToKernel(cyjsWidget);
               });
            } // ready
           })},

        updateSelectionToKernel: function(cyjsWidget){
           console.log("*** entering updateSelectionToKernel") 
           var selectedNodes = cyjsWidget.cy.nodes(":selected");
           var selectedNodeIDs = selectedNodes.map(function(n){return (n.data("name") )})
           var selectedEdges = cyjsWidget.cy.edges(":selected");
           var selectedNodeCount = selectedNodes.length;
           var selectedEdgeCount = selectedEdges.length;
           console.log("selected nodes: " + selectedNodeCount);
           console.log("selected edGes:" + selectedEdgeCount);
           var jsonString = JSON.stringify({cmd: "updateSelectedNodes",
                                            status: "request",
                                            callback: "",
                                            payload: selectedNodeIDs});
           console.log(" *** jsonString: ")
           console.log(jsonString);
           cyjsWidget.model.set("msgFromBrowser", jsonString);
           console.log("    after setting 'msgFromBrowser");
           cyjsWidget.touch();
           },
        
        loadStyle: function(filename){
           var cyObj = this.cy;
           console.log("cyjsWidget.loadStyle: " + filename)
           console.log("loadStyle's notion of this:");
           console.log(this);
           console.log("loadStyle's notion of cy:");
           console.log(cyObj);
           var str = window.location.href;
           var url = str.substr(0, str.lastIndexOf("/")) + "/" + filename;
           url = url.replace("/notebooks/", "/files/");
           console.log("about to getScript: " + url);
           $.getScript(url)
              .done(function(script, textStatus) {
                 console.log(textStatus);
                 cyObj.style(vizmap);
                 })
             .fail(function( jqxhr, settings, exception ) {
                console.log("getScript error trying to read " + filename);
                console.log("exception: ");
                console.log(exception);
                });
          },
        
        loadGraph: function(filename){
           console.log("entering loadGraph");
           var cyObj = this.cy;
              // the robust url of a file in the same directory as the notebook is
              // str.substring(0, str.lastIndexOf("/"));
           var str = window.location.href;
           var url = str.substr(0, str.lastIndexOf("/")) + "/" + filename;
           url = url.replace("/notebooks/", "/files/");
           console.log("=== about to getScript on " + url);
           $.getScript(url)
              .done(function(script, textStatus) {
                 console.log("getScript: " + textStatus);
                 console.log("nodes: " + network.elements.nodes.length);
                 if(typeof(network.elements.edges) != "undefined")
                    console.log("edges: " + network.elements.edges.length);
                cyObj.add(network.elements);  // no positions yet
                cyObj.nodes().map(function(node){node.data({degree: node.degree()})});
                cyObj.layout({"name": "grid"});
                cyObj.fit(150);
                }) // .done
            .fail(function(jqxhr, settings, exception) {
               console.log("addNetwork getscript error trying to read " + filename);
               });
           },
        
        render: function() { 
            console.log("entering render")
            this.$el.append(this.createDiv());
            this.listenTo(this.model, 'change:frameHeight', this.frameDimensionsChanged, this);
            this.listenTo(this.model, 'change:msgFromKernel', this.dispatchRequest, this);
            var cyjsWidget = this;
            function myFunc(){
               cyjsWidget.createCanvas()
               }
            setTimeout(myFunc, 500);
            },

        setNodeAttributes: function(attributeName, nodeNames, values){
          console.log("entering setNodeAttributes")
          console.log("have " + nodeNames.length + " values for attribute " + attributeName);
          for(var i=0; i < nodeNames.length; i++){
             var name = nodeNames[i];
             var newValue = values[i];
             var filterString = "[name='" + name + "']";
             console.log("filterString: " + filterString);
             console.log("nodeName: " + name + "   value: " + newValue);
             var dataObj = this.cy.nodes().filter(filterString).data();
             Object.defineProperty(dataObj, attributeName, {value: newValue});
             }// for i
         }, // setNodeAttributes
        
        setEdgeAttributes: function(attributeName, sourceNodes, targetNodes, edgeTypes, attributeValues){
            for(var i=0; i < sourceNodes.length; i++){
              var selectorString = "edge[source='" + sourceNodes[i] + "'][target='" + targetNodes[i] +
                                   "'][edgeType='" + edgeTypes[i] + "']";
              console.log(selectorString);
              console.log("eda value: " + attributeValues[i]);
              var dataObj = cy.edges().filter(selectorString).data();
              if(dataObj != undefined){
                 Object.defineProperty(dataObj, attributeName, 
                                       {value: attributeValues[i], configurable: true});
                 }                  
             } // for i
          }, // setEdgeAttributes
       
        dispatchRequest: function(){
           console.log("dispatchRequest");
           var msgRaw = this.model.get("msgFromKernel");
           var msg = JSON.parse(msgRaw);
           console.log(msg);
           console.log("========================");
           console.log(this);
           switch(msg.cmd) {
              case 'deleteGraph':
                this.cy.edges().remove();
                this.cy.nodes().remove();
                break;
              case 'addGraph':
                 var g = msg.payload;
                 console.log("addGraph request")
                 console.log(g)
                 this.cy.add(g.elements)
                 break;
              case 'layout':
                 var layoutName = msg.payload;
                 this.cy.layout({"name": layoutName})
                 break;
              case 'setPosition':
                var positionObjects = msg.payload;
                console.log("calling setPosition map");
                positionObjects.map(function(e){
                  var tag="[id='" + e.id + "']";
                  cy.$(tag).position({x: e.x, y:e.y});
                  });
                break;
              case 'setNodeAttributes':
                var attributeName = msg.payload.attributeName;
                var nodeNames = msg.payload.nodeNames;
                var values = msg.payload.values;
                this.setNodeAttributes(attributeName, nodeNames, values);
                this.cy.style().update();
                break;
                   
              case 'setEdgeAttributes':
                var attributeName = msg.payload.attributeName;
                var sourceNames = msg.payload.sourceNames;
                var targetNames = msg.payload.targetNames;
                var edgeTypes = msg.payload.edgeTypes;
                var attributeValues = msg.payload.values;
                console.log("js setEdgeAttributes, count = " + sourceNames.length);
                if(typeof(sourceNames) == "string") sourceNames = [sourceNames];
                if(typeof(targetNames) == "string") targetNames = [targetNames];
                if(typeof(edgeTypes) == "string") edgeTypes = [edgeTypes];
                this.setEdgeAttributes(attributeName, sourceNames, targetNames, edgeTypes, attributeValues);
                this.cy.style().update()
                break;
                   
              case 'loadStyleFile':
                var styleFile = msg.payload;
                this.loadStyle(styleFile);
                break;
              case 'fit':
                 var margin = msg.payload;
                 console.log("fit with margin: " + margin)
                 this.cy.fit(margin);
                 break;

               case 'fitSelected':
                 var margin = msg.payload;
                 console.log("fit with margin: " + margin)
                 this.cy.fit(this.cy.nodes(":selected"), margin);
                 break;

              case 'getSelectedNodes':
                 var selectedNodes = this.cy.filter("node:selected").map(function(node){ 
                     return node.data().id});
                  console.log("-- found these selected nodes: ");
                  console.log(selectedNodes);
                  var jsonString = JSON.stringify({cmd: "storeSelectedNodes",
                                                status: "reply",
                                                callback: "",
                                                payload: selectedNodes})
                  console.log(" *** jsonString: ")
                  console.log(jsonString);
                  this.model.set("msgFromBrowser", jsonString);
                  console.log("    after setting 'msgFromBrowser");
                  this.touch();
                  break;
               case 'selectNodes':
                  var nodeNames = msg.payload;
                  console.log("--- selecting these nodes: " + nodeNames);
                  if(typeof(nodeNames) == "string")
                     nodeNames = [nodeNames];
                 var filterStrings = [];
                 for(var i=0; i < nodeNames.length; i++){
                   var s = '[name="' + nodeNames[i] + '"]';
                   filterStrings.push(s);
                   } // for i
                var nodesToSelect = this.cy.nodes(filterStrings.join());
                nodesToSelect.select()
                break;
              case 'clearSelection':
                 this.cy.nodes().unselect();
                 break;
            default:
               console.log("unrecognized msg.cmd: " + msg.cmd);
             } // switch
           console.log("CONCLUDING dispatchRequest")
           }, 
        
        frameDimensionsChanged: function(){
           console.log("frameDimensionsChanged");
           var newHeight = this.model.get("frameHeight");
           console.log("frameHeight: "  + newHeight);
           $("#cyOuterDiv").height(newHeight);
           this.cy.resize();
           // $("#cyDiv").height(newHeight - $("#cyToolbarDiv").height());
           }, 
        
        events: {
           //"click #svg": "changeHandler"
           }

    });
    return {
        CyjsView: CyjsView
    };
});

<IPython.core.display.Javascript object>

In [38]:
cy = cyjs_sa()
display(cy)

In [51]:
cy.setHeight(800)

In [39]:
from igraph import *
import random

networkFile = "toyNetwork2.tsv"
nodeAttributesFile = "type.noa"
lines = open(networkFile).readlines()
tbl_rel = []
for line in lines:
   tokens = line.rstrip("\n").split("\t")
   tokens = [x.strip() for x in tokens]
   if(len(tokens) == 3):
      tbl_rel.append(tokens)



In [40]:
lines = open(nodeAttributesFile).readlines()

nodeTypes=[]
for line in lines:
   (node, type) = line.rstrip("\n").split("\t")
   nodeTypes.append([node, type])

g = Graph(directed=True)
nodeNames = [x[0] for x in nodeTypes]
nodeTypeValues = [x[1] for x in nodeTypes]

g.add_vertices(nodeNames)
g.vs['type'] = nodeTypeValues


In [41]:
for row in tbl_rel:
  source = row[0];
  target = row[1];
  #print("adding edge, %s -> %s" % (source, target))
  g.add_edge(source, target)

g.es['edgeType'] = [x[2] for x in tbl_rel]
#g.es()['flux'] = random.sample(range(0, 1000), len(g.es()))


In [42]:
g.es()[0].target

5

In [43]:
g

In [44]:
cy.deleteGraph()
cy.addGraph(g)

In [45]:
cy.loadStyleFile("style2.js")

In [46]:
cy.selectNodes("R3")
cy.fitSelected(200)

In [47]:
rtLayout = g.layout("rt")
cy.setPosition(rtLayout)
kkLayout = g.layout("kk")
cy.setPosition(kkLayout)
cy.fit(100)
#for i in range(len(rtLayout)):
#   print(rtLayout[i])


In [157]:
igraphLayoutStrategies = ["circle", "drl", "fruchterman_reingold", 
                          "fruchterman_reingold_3d", 
                          "kamada_kawai", "kamada_kawai_3d", "lgl",
                          "random", "random_3d", "reingold_tilford",
                          "reingold_tilford_circular", "sphere"]

In [158]:
#for strategy in igraphLayoutStrategies:
#   print("---- %s" % strategy)
#   layout = g.layout(strategy)
#   cy.setPosition(layout)
#   cy.fit(100)
#   time.sleep(1)

In [68]:
cy.getSelectedNodes()

['R3']

In [69]:
cy.clearSelection()

In [22]:
from pandas import *
tbl_flux = DataFrame.from_csv('flux.tsv', sep='\t', header=0)
tbl_flux

/Users/paul/anaconda/lib/python3.5/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory: 'sysctl'
  stacklevel=stacklevel + 1):


,cond1,cond2,cond3
R1,477,246,752
R2,44,734,633
R3,799,703,247
R4,368,736,551
R5,563,801,234


In [23]:
reactionNames = tbl_flux.index.tolist()
conditionNames = tbl_flux.columns.tolist()

In [24]:
values = tbl_flux[conditionNames[0]].tolist()
values

[477, 44, 799, 368, 563]

In [25]:
cy.setNodeAttributes("flux", reactionNames, values)

In [26]:
tbl_edgeFlux = read_csv('edgeFlux.tsv', sep='\t')
tbl_edgeFlux

,source,target,edgeType,cond1,cond2,cond3
0,R1,a,consumes,701,369,691
1,R1,b,consumes,885,389,556
2,R1,c,produces,274,692,927
3,R2,a,consumes,326,364,181
4,R2,d,consumes,742,90,643
5,R2,e,produces,832,660,96
6,R3,c,consumes,83,275,394
7,R3,f,consumes,318,224,587
8,R3,h,produces,312,448,170
9,R3,i,produces,661,94,7


In [27]:
sourceNodes = tbl_edgeFlux["source"].tolist()
targetNodes = tbl_edgeFlux["target"].tolist()
edgeTypes = tbl_edgeFlux["edgeType"].tolist()
values = tbl_edgeFlux["cond3"]

In [28]:
cy.setEdgeAttributes("flux", sourceNodes, targetNodes, edgeTypes, values)

In [184]:
cy.selectNodes(["R3", "i"])

In [185]:
cy.getSelectedNodes()

[]

In [34]:
cy.fitSelected(200)

In [222]:
len(g.vs)

26

In [225]:
g.vs[1]

igraph.Vertex(<igraph.Graph object at 0x104b42318>, 1, {'name': 'R2', 'type': 'reaction'})

In [46]:
g.vs.find("R5").index

4

In [66]:
names = g.vs['name']
ids = [g.vs.find(name).index for name in g.vs['name']]
nodeMap = dict(zip(names, ids))

In [67]:
nodeMap["R3"]

2

In [69]:
nodes = ["R1", "c", "R5", "g1"]
[nodeMap[name] for name in nodes]

[0, 7, 4, 11]

In [45]:
[g.vs.find(x).index for x in g.vs['name']]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

In [242]:
x

{'R1': 0,
 'R2': 1,
 'R3': 2,
 'R4': 3,
 'R5': 4,
 'a': 5,
 'b': 6,
 'c': 7,
 'd': 8,
 'e': 9,
 'f': 10,
 'g1': 11,
 'g2': 12,
 'g3': 13,
 'g4': 14,
 'g5': 15,
 'g6': 16,
 'h': 17,
 'i': 18,
 'orphan1': 19,
 'orphan2': 20,
 'w': 21,
 'x': 22,
 'y': 23,
 'z': 24,
 'z1': 25}